In [5]:
import scipy.io
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

In [6]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("CUDA Available:", torch.cuda.is_available())
print("Device Count:", torch.cuda.device_count())
print("Current Device Index:", torch.cuda.current_device())
print("Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No CUDA device found")

CUDA Available: False
Device Count: 0


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
# Load the dataset
data = scipy.io.loadmat("ex4data1.mat")
X = data["X"]
y = data["y"].flatten()
y[y == 10] = 0  # Replace label 10 with 0 to match PyTorch expectations (0-9 classes)

In [ ]:
# Convert data to torch tensors
X = torch.tensor(X, dtype=torch.float32).to(device)
y = torch.tensor(y, dtype=torch.long).to(device)

In [ ]:
# Dataset and DataLoader
dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=64, shuffle=True)

In [ ]:
# Define Neural Network
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.sigmoid(self.layer1(x))
        x = torch.sigmoid(self.layer2(x))  # Sigmoid for consistency with original code
        return x

In [ ]:
# Hyperparameters
input_size = 400
hidden_size = 25
output_size = 10
learning_rate = 0.8
num_epochs = 50
lambda_reg = 1.0

In [ ]:
# Initialize model, loss, and optimizer
model = NeuralNet(input_size, hidden_size, output_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=lambda_reg)

In [ ]:
# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {running_loss / len(loader):.4f}")

In [ ]:
# Prediction and Accuracy
with torch.no_grad():
    outputs = model(X)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y).float().mean() * 100
    print(f"Training Set Accuracy: {accuracy:.2f}%")